## Tests for tof_functions

In [1]:
import os
import numpy                 as np
import pandas                as pd
import hypothesis.strategies as st

from hypothesis  import given

import antea.reco.reco_functions as rf
import antea.elec.tof_functions  as tf
from antea.io.mc_io import load_mcTOFsns_response

### spe_dist function

In [2]:
tau_sipm = (100, 15000)
time     = np.arange(10)
exp_dist = tf.spe_dist(tau_sipm, time)

assert len(exp_dist) == len(time)
assert (exp_dist >= 0.).all()
assert np.sum(exp_dist) == 1

In [3]:
time = np.array([1, 1491])
alfa         = 1.0/2
beta         = 1.0/1
if np.isclose(alfa, beta, rtol=1e-2):
    spe_response = np.zeros(len(time))
t_p          = np.log(beta/alfa)/(beta-alfa)
K            = (beta)*np.exp(alfa*t_p)/(beta-alfa)
spe_response = K*(np.exp(-alfa*time)-np.exp(-beta*time))
print(np.exp(-alfa*time))
print(np.exp(-beta*time))
print(np.exp(-alfa*time)-np.exp(-beta*time))
print(K*(np.exp(-alfa*time)-np.exp(-beta*time)))
print(spe_response/np.sum(spe_response))
spe_response = spe_response/np.sum(spe_response) #Normalization
print(spe_response)

np.isclose(alfa, beta, rtol=1e-2)# or 
np.count_nonzero(np.exp(-(alfa)*time))# and ((np.exp(-(1/b)*l)).all()==0)):
#np.exp(-(alfa)*time)


[0.60653066 0.        ]
[0.36787944 0.        ]
[0.23865122 0.        ]
[0.95460487 0.        ]
[1. 0.]
[1. 0.]


1

In [4]:
def spe_dist(tau_sipm, time):
    alfa         = 1.0/tau_sipm[1]
    beta         = 1.0/tau_sipm[0]
    if np.isclose(alfa, beta, rtol=1e-2):
        return np.zeros(len(time))
    t_p          = np.log(beta/alfa)/(beta-alfa)
    K            = (beta)*np.exp(alfa*t_p)/(beta-alfa)
    spe_response = K*(np.exp(-alfa*time)-np.exp(-beta*time))
    if np.sum(spe_response) == 0:
        return np.zeros(len(time))
    spe_response = spe_response/np.sum(spe_response) #Normalization
    return spe_response

In [5]:
#a = st.floats(min_value=1, max_value=1)
#b = st.floats(min_value=2, max_value=2)
#l = st.lists(st.floats(min_value=1, max_value=1), st.floats(min_value=1491, max_value=1491))

a = st.floats(min_value=1, max_value=100000)
b = st.floats(min_value=2, max_value=100000)
l = st.lists(st.integers(min_value=1, max_value=10000), min_size=2, max_size=1000)

@given(a, b, l)
def test_spe_dist(a, b, l):
    l = np.array(l)
    exp_dist = tf.spe_dist((a, b), np.unique(l))
    count_a = np.count_nonzero(np.exp(-(1/a)*time))
    count_b = np.count_nonzero(np.exp(-(1/b)*time))
    if np.isclose(1/a, 1/b, rtol=1e-2) or (not count_a and not count_b):
        assert np.count_nonzero(exp_dist) == 0
        assert np.isclose(np.sum(exp_dist), 0)
    else:
        assert len(exp_dist) == len(np.unique(l))
        assert (exp_dist >= 0.).all()
        assert np.isclose(np.sum(exp_dist), 1)

In [6]:
test_spe_dist()

In [7]:
np.count_nonzero(np.array([0.001, 0.2, 2, 0.0005, 0]))
np.nonzero([0.001, 0.2, 2, 0.0005, 0])

(array([0, 1, 2, 3]),)

### convolve_tof function

In [8]:
def convolve_tof(spe_response, signal):
    if not np.count_nonzero(spe_response):
        print('Distrib values are zero')
        return np.zeros(len(spe_response)+len(signal)-1 )
    conv_first = np.hstack([spe_response, np.zeros(len(signal)-1)])
    conv_res   = np.zeros(len(signal)+len(spe_response)-1)
    pe_pos     = np.argwhere(signal > 0)
    pe_recov   = signal[pe_pos]
    for i in range(len(pe_recov)): #Loop over the charges
        conv_first_ch = conv_first*pe_recov[i]
        desp          = np.roll(conv_first_ch, pe_pos[i])
        conv_res     += desp
    return conv_res

In [9]:
#t = st.tuples(st.floats(min_value=1, max_value=1000), st.floats(min_value=1, max_value=1000))
#l = st.lists(st.integers(min_value=1, max_value=10000), min_size=2, max_size=1000)
#s = st.lists(st.integers(min_value=1, max_value=10000), min_size=2, max_size=1000)

t = st.tuples(st.floats(min_value=1, max_value=1), st.floats(min_value=2, max_value=2))
l = st.lists(st.integers(min_value=1491, max_value=1491), min_size=2, max_size=2)
s = st.lists(st.integers(min_value=1, max_value=1), min_size=2, max_size=2)

@given(t, l, s)
def test_convolve_tof(t, l, s):
    spe_response = tf.spe_dist(t, np.unique(np.array(l)))
    print(spe_response)
    conv_res     = convolve_tof(spe_response, np.array(s))
    assert len(conv_res) == len(spe_response) + len(s) - 1
    if np.count_nonzero(spe_response):
        print(' nooon zeeero')
        assert np.isclose(np.sum(s), np.sum(conv_res))

In [10]:
test_convolve_tof()

[0.]
Distrib values are zero


### tdc convolution function:

In [11]:
def tdc_convolution(tof_response, spe_response, time_window, n_sipms, first_sipm, te_tdc):
    """
    Apply the spe_response distribution to every sipm and returns a charge matrix of time and n_sipms dimensions.
    """
    pe_table  = np.zeros((time_window, n_sipms))
    for i, wf in tof_response.iterrows():
        if wf.time_bin < time_window:
            s_id = - wf.sensor_id - first_sipm
            pe_table[wf.time_bin, s_id] = wf.charge

    conv_table = np.zeros((len(pe_table) + len(spe_response)-1, n_sipms))
    for i in range(n_sipms):
        if np.count_nonzero(pe_table[:,i]):
            conv_table[:,i] = convolve_tof(spe_response, pe_table[0:time_window,i])
    return conv_table

In [ ]:
PATH_IN      = '/Users/carmenromoluque/nexus_petit_analysis/PETit-ring/change_to_dataframes/full_ring_iradius165mm_depth3cm_pitch7mm_new_h5.001.pet.h5'
PATH_IN2     = '/Users/carmenromoluque/ANTEA/antea/testdata/ring_test_1000ev.h5'
DataSiPM     = db.DataSiPM('petalo', 0)
DataSiPM_idx = DataSiPM.set_index('SensorID')

sns_response = pd.read_hdf(PATH_IN, 'MC/waveforms')
tof_response = pd.read_hdf(PATH_IN, 'MC/tof_waveforms')
threshold    = 2
charge_range = (1000, 1400)
sel_df       = rf.find_SiPMs_over_threshold(sns_response, threshold)

particles = pd.read_hdf(PATH_IN, 'MC/particles')
hits      = pd.read_hdf(PATH_IN, 'MC/hits')
events    = particles.event_id.unique()

for evt in events[831:832]:
    sns     = sel_df[sel_df.event_id==evt]
    chargs  = sns[sns.charge==sns.charge]
    sensors = DataSiPM.loc[chargs.sensor_id]
    max_sns = sns[sns.charge == sns.charge.max()]
    if len(max_sns != 1):
        max_sns = max_sns[max_sns.sensor_id == max_sns.sensor_id.min()]
    max_sipm  = DataSiPM_idx.loc[max_sns.sensor_id]
    max_pos   = np.array([max_sipm.X.values, max_sipm.Y.values, max_sipm.Z.values]).transpose()[0]
    sel_parts = particles[particles.event_id==evt]
    sel_hits  = hits[hits.event_id==evt]
    
    reconstruct_coincidences(sns_response, tof_response, charge_range, DataSiPM_idx, particles, hits)